<a href="https://colab.research.google.com/github/arifpras/tfda-djppr/blob/main/20250619_fpl_fwd_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive  # Import the drive module from google.colab so we can access Google Drive

drive.mount('/content/drive')   # Mount (connect) your Google Drive to the path /content/drive so files can be accessed

In [ ]:
setwd("/content/drive/MyDrive/00fpl")  # Set the working directory to the specified folder in your Google Drive

getwd()                                # Show the current working directory to confirm the change

In [ ]:
library(readr)   # Load the 'readr' package, which provides functions to read data (like CSV files)

library(dplyr)   # Load the 'dplyr' package, useful for data manipulation (filtering, selecting, grouping, etc.)

db00 <- read_csv("/content/drive/MyDrive/00fpl/db00all_fpl.csv", show_col_types = FALSE)
# Read the CSV file from your Google Drive into a data frame called 'db00'
# 'show_col_types = FALSE' hides the column type message

# db00 <- read_csv("https://raw.githubusercontent.com/arifpras/tfda-djppr/refs/heads/main/courses/workshop01/db00all_fpl.csv", show_col_types = FALSE)
# (Optional alternative) Load the same CSV file directly from a GitHub URL instead of Google Drive

In [ ]:
glimpse(db00)  # Quickly display the structure of the 'db00' data frame: column names, types, and example values

In [ ]:
important_vars <- c(
  "name", "total_points", "position", "team", "x_p", "assists", "bonus", "bps",
  "clean_sheets", "creativity", "expected_assists", "expected_goal_involvements",
  "expected_goals", "expected_goals_conceded", "goals_conceded", "goals_scored",
  "influence", "minutes", "own_goals", "penalties_missed",
  "penalties_saved", "red_cards", "saves", "selected", "starts", "team_a_score",
  "team_h_score", "threat", "transfers_balance", "value"
)
# Define a list of important columns to keep from the dataset

db01 <- db00 %>%
  select(all_of(important_vars)) %>%              # Keep only the specified columns from db00
  filter(minutes != 0, position == "FWD", value >= 36) %>%  # Keep only players with non-zero minutes, who play as forwards, and have value >= 36
  mutate(value = value / 10) %>%                  # Convert player value to original scale (e.g., 100 → 10.0)
  relocate(total_points)                          # Move 'total_points' column to the front
# Create a new filtered and cleaned dataset called db01

glimpse(db01)  # Display the structure of db01: column names, types, and example values

In [ ]:
db01 %>% head()  # Show the first 6 rows of the cleaned dataset 'db01' to get a quick look at the data

In [ ]:
db01 %>%
  arrange(desc(total_points)) %>%                     # Sort the players from highest to lowest total points
  select(name, team, position, value, total_points)   # Select only key columns to display: name, team, position, value, and total points

In [ ]:
# install.packages("colorspace")  # (Run once if not yet installed) Installs the 'colorspace' package for color customization

library(ggplot2)     # Load ggplot2 for plotting
library(colorspace)  # Load colorspace for advanced color manipulation
library(dplyr)       # Load dplyr for data manipulation

# Filter and plot
db01 %>%
  select(-name) %>%  # Exclude the 'name' column to simplify the plot
  ggplot(aes(x = team, y = total_points)) +  # Start a ggplot: team on x-axis (flipped later), total points on y-axis
  geom_boxplot(
    aes(
      fill = after_scale(desaturate(lighten(color, 0.7), 0.7))  # Use a lightened and desaturated version of the default fill color
    ),
    size = 1,      # Thickness of boxplot lines
    color = "grey50"  # Border color of boxplots
  ) +
  scale_fill_manual(values = NULL) +  # Allow manual fill scale (required when using after_scale, even if NULL)
  # facet_wrap(~ obsvar, scales = "free", nrow = 1) +  # (Optional) Facet if you'd like to split by another variable
  theme_light() +  # Use a clean light theme
  labs(
    title = "Boxplot of Total Points by Team (FWD)",  # Plot title
    x = "\nTeam",  # X-axis label
    y = "",        # No Y-axis label
    color = NULL   # No legend title
  ) +
  coord_flip() +  # Flip the coordinates to make team names readable (horizontal boxplots)
  theme(
    axis.text.x = element_text(size = 8),           # Customize x-axis text size
    axis.ticks.x = element_blank(),                 # Remove x-axis ticks
    axis.line.x = element_blank(),                  # Remove x-axis lines
    axis.title.x = element_text(size = 7),          # X-axis title size
    axis.text.y = element_text(size = 8),           # Y-axis (team names) text size
    axis.title.y = element_text(size = 7),          # Y-axis title size
    axis.line.y = element_blank(),                  # Remove y-axis lines
    plot.title = element_text(hjust = 0, size = 16, face = "bold"),  # Title aligned left, bold, larger size
    plot.title.position = "plot",                   # Title position
    strip.text.x = element_text(size = 8),          # Facet strip label size (if faceting)
    panel.grid.major.y = element_line(color = "grey90"),  # Light gridlines for readability
    panel.spacing = unit(1, "lines"),               # Spacing between facets (if used)
    legend.position = "none"                        # Hide legend
  )

In [ ]:
db01 %>%
  group_by(team) %>%  # Group the data by 'team'

  summarise(
    count_players = n(),                                    # Number of forwards in each team
    min_points = min(total_points, na.rm = TRUE),           # Minimum total points
    q1 = quantile(total_points, 0.25, na.rm = TRUE),        # 25th percentile (Q1)
    q2_median = median(total_points, na.rm = TRUE),         # Median (Q2)
    q3 = quantile(total_points, 0.75, na.rm = TRUE),        # 75th percentile (Q3)
    q4 = max(total_points, na.rm = TRUE),                   # Maximum value (same as max_points, redundant here)
    max_points = max(total_points, na.rm = TRUE),           # Maximum total points (repeated for clarity)
    mean_points = mean(total_points, na.rm = TRUE),         # Mean total points
    value_weighted_avg = weighted.mean(total_points, w = value, na.rm = TRUE)  # Value-weighted average of total points
  ) %>%
  arrange(desc(q2_median))  # Sort the results by median total points (highest team medians first)

In [ ]:
install.packages("stargazer")  # (Run once) Install the 'stargazer' package for beautiful summary tables

library(stargazer)  # Load the stargazer package

db01 %>%
  as.data.frame() %>%                            # Convert tibble to standard data frame (stargazer prefers base data frames)
  stargazer(type = 'text',                      # Output the summary as plain text (other options: 'html', 'latex')
            out = "descsumm01_fwd.txt",         # Save the summary output to a text file named "descsumm01_fwd.txt"
            digits = 1)                         # Round all numeric summaries to 1 decimal place

In [ ]:
install.packages("parameters")  # (Run once) Install the 'parameters' package to compute advanced descriptive statistics

library(dplyr)       # Load dplyr for data manipulation
library(parameters)  # Load parameters for descriptive summary functions

# Get the summary as a data frame
descsumm02 <- db01 %>%
  select(where(is.numeric)) %>%      # Select only numeric columns from db01
  describe_distribution()            # Generate descriptive stats (mean, SD, skewness, kurtosis, etc.)

descsumm02  # View the summary table

# Save it as plain text
capture.output(
  print(descsumm02, digits = 1),     # Format the output with 1 decimal precision
  file = "descsumm02_fwd.txt"        # Save the printed summary to a file named "descsumm02_fwd.txt"
)

In [ ]:
install.packages("corrr")  # (Run once) Install the 'corrr' package to compute and format correlation matrices

library(corrr)  # Load the corrr package

corr01 <- db01 %>%
  select(where(is.numeric)) %>%  # Keep only numeric columns from db01
  correlate() %>%                # Compute pairwise correlations between all numeric variables
  # shave() %>%                 # (Optional) Remove the upper triangle of the correlation matrix
  fashion()                     # Format the correlation matrix for cleaner display (aligns numbers, adds spacing)

corr01  # Display the formatted correlation table

# Save the output as a plain text file
capture.output(print(corr01), file = "corr01_fwd.txt")  # Write the result to "corr01_fwd.txt"

In [ ]:
db01 %>%
  select(where(is.numeric)) %>%         # Select only numeric columns from db01
  correlate() %>%                        # Compute pairwise correlations between numeric variables
  as.data.frame() %>%                    # Convert the correlation matrix to a regular data frame
  write.csv("corr01_fwd.csv", row.names = FALSE)  # Save it as a CSV file without row names

In [ ]:
install.packages("viridis")  # (Run once) Install the 'viridis' package for colorblind-friendly color scales

library(corrr)     # For calculating correlations
library(dplyr)     # For data manipulation
library(tidyr)     # For reshaping data
library(ggplot2)   # For plotting
library(viridis)   # For advanced color palettes (colorblind-friendly)

# Step 1: Compute correlations
corr_matrix <- db01 %>%
  select(where(is.numeric)) %>%  # Select only numeric columns
  correlate()                    # Compute correlation matrix

# Step 2: Reshape to long format (for ggplot heatmap)
corr_long <- corr_matrix %>%
  pivot_longer(-term, names_to = "variable", values_to = "correlation")  # Convert wide matrix to long format

# Step 3: Create heatmap plot object
corr_plot <- ggplot(corr_long, aes(x = term, y = variable, fill = correlation)) +
  geom_tile(color = "white") +                                   # Create colored squares for each correlation
  scale_fill_viridis_c(option = "D", limits = c(-1, 1), name = "Correlation") +  # Apply viridis color scale from -1 to 1
  # coord_fixed() +                                                # Fix aspect ratio for square tiles
  theme_minimal() +                                              # Use a clean minimal theme
  theme(
    axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1),  # Rotate x-axis labels for readability
    axis.text.y = element_text(size = 8),
    panel.grid = element_blank(),
    plot.title = element_text(size = 14, face = "bold", hjust = 0),
    legend.title = element_text(size = 9),
    legend.text = element_text(size = 8)
  ) +
  labs(title = "Correlation Heatmap", x = "", y = "")  # Set title and remove axis labels

corr_plot  # Display the heatmap in the notebook

# Step 4: Save to PDF
ggsave("heatmap01_fwd.pdf", plot = corr_plot, width = 11.7, height = 8.3)  # Export the plot to a landscape A4 PDF

In [ ]:
# library(tidyverse)  # (Optional) Load the full tidyverse suite if needed
library(tidyr)        # Load tidyr for data reshaping functions like pivot_longer()

db02 <- db01 %>%
  select(-name) %>%                  # Remove the 'name' column to simplify the reshaped data
  relocate(team, position) %>%      # Move 'team' and 'position' columns to the front
  pivot_longer(                     # Reshape the data from wide to long format:
    cols = total_points:value,      # Convert all columns from 'total_points' to 'value' into key-value pairs
    names_to = "obsvar",            # Store original column names in a new column called 'obsvar'
    values_to = "obsval"            # Store the corresponding values in a column called 'obsval'
  )

glimpse(db02)  # Display the structure of the reshaped dataset

In [ ]:
important_vars <- c(
  "total_points", "team", "assists", "creativity",
  "expected_assists", "expected_goals", "goals_scored", "influence",
  "minutes", "own_goals", "penalties_missed", "selected", "starts",
  "threat", "transfers_balance", "value"
)
# Define a list of important variables (column names) to include in the new dataset

db03 <- db01 %>%
  select(all_of(important_vars)) %>%  # Select only the specified variables from db01
  relocate(total_points)              # Move 'total_points' column to the front for easier reference

glimpse(db03)  # Display the structure of the new dataset (column types and example values)

In [ ]:
db03$team <- factor(db03$team)  # Convert the 'team' column to a factor (categorical variable)

db03$team <- relevel(db03$team, ref = "Man City")
# Set "Man City" as the reference category for the 'team' factor.
# This means all other team coefficients in the regression will be compared to "Man City"

ols_fwd_base <- lm(total_points ~ ., data = db03)
# Fit a linear regression model predicting total_points using all other variables in db03
# (The "." means "use all remaining columns as predictors")

# ols_fwd_robust <- lm_robust(
#   total_points ~ ., data = db03, se_type = "stata"
# )
# (Optional) Robust version using heteroskedasticity-consistent standard errors (commented out)

summary(ols_fwd_base)  # Print the summary of the OLS regression model

# summary(ols_fwd_robust)
# Print the robust regression summary (if using the lm_robust version)

In [ ]:
# Load required libraries
# install.packages("glmnet")  # (Run once if not installed)
library(glmnet)  # For regularized regression like LASSO
library(dplyr)   # For data manipulation

# Prepare data
# (Uncomment and adjust if you want a custom version of db_lasso)
# db_lasso <- db01 %>%
#   select(total_points, team, x_p, assists, bonus, bps, clean_sheets, creativity,
#          expected_assists, expected_goals, goals_scored, influence,
#          minutes, own_goals, penalties_missed, selected, starts, threat,
#          transfers_balance, value) %>%
#   na.omit()  # Remove rows with missing values

# Convert categorical variable 'team' to dummy variables
x <- model.matrix(total_points ~ ., data = db03)[, -1]  # Create matrix of predictors (excluding intercept column)
y <- db_lasso$total_points  # Target variable (total points)

# ⚙️ Fit LASSO using cross-validation
set.seed(123)  # For reproducibility
cv_fit <- cv.glmnet(x, y, alpha = 1, standardize = TRUE)
# Perform LASSO (alpha = 1) with automatic lambda tuning via cross-validation

# Optimal lambda
best_lambda <- cv_fit$lambda.min  # Best lambda minimizing mean cross-validated error
cat("Best lambda:", best_lambda, "\n")

# Plot cross-validation error curve
plot(cv_fit)  # Shows how model error changes as lambda increases

# Extract coefficients at best lambda
coef(cv_fit, s = "lambda.min")  # Show which variables are kept/shrunk

# Predict fitted values (optional)
pred <- predict(cv_fit, newx = x, s = "lambda.min")  # Make predictions using the selected lambda

What this does:

- Automatically selects the most predictive variables for total_points using LASSO regularization.
- Shrinks less important coefficients to zero — great for feature selection.
- Uses cross-validation to find the optimal penalty (lambda.min).

In [ ]:
# Assuming `lasso_coef` is your sparse matrix from coef(cv_fit, s = "lambda.min")
lasso_coef <- coef(cv_fit, s = "lambda.min")
# Extract the coefficients from the best LASSO model (based on lambda.min)

# Convert to a tidy data frame
coef_df <- as.matrix(lasso_coef) %>%                  # Convert the sparse matrix to a regular matrix
  as.data.frame() %>%                                 # Convert matrix to data frame
  tibble::rownames_to_column(var = "feature") %>%     # Move row names (variable names) into a column called "feature"
  rename(coefficient = s0)                            # Rename the column holding coefficients to "coefficient"

# Optional: filter only non-zero coefficients
coef_df_nonzero <- coef_df %>%
  filter(coefficient != 0)  # Keep only the features selected by the LASSO (i.e., those with non-zero coefficients)

# View
print(coef_df_nonzero)  # Display the non-zero coefficients and their associated features

In [ ]:
# install.packages("estimatr")  # (Run once) Install for robust regression tools
library(estimatr)  # Load package (used if running lm_robust)

# install.packages("modelsummary")  # (Optional) Install for pretty regression tables
# library(modelsummary)  # Load modelsummary (used if you want clean tables with stars)

# 💡 Optional robust version using lm_robust (commented out)
# ols_fwd_lasso <- lm_robust(
#     total_points ~ team + assists + creativity + expected_assists + expected_goals +
#       goals_scored + influence + minutes + own_goals + penalties_missed + starts +
#       threat + transfers_balance + value, data = db03, se_type = "stata")

# Basic OLS regression using only variables selected by LASSO
ols_fwd_lasso <- lm(
  total_points ~ team + assists + creativity + expected_assists + expected_goals +
    goals_scored + influence + minutes + own_goals + penalties_missed + starts +
    threat + transfers_balance + value, data = db03)

# View summary of the model: coefficients, R², significance levels, etc.
summary(ols_fwd_lasso)

# (Optional) Generate a clean regression table with significance stars
# modelsummary(ols_fwd_lasso, stars = TRUE)

This version runs an OLS regression using only the LASSO-retained predictors, helping you evaluate their statistical significance more traditionally (p-values, R², etc.).

In [ ]:
# ols_fwdsw <- step(ols_fwd_base, direction = "both")
ols_fwdsw <- step(ols_fwd_base, direction = "both")
# Perform stepwise regression starting from the full model (ols_fwd_base)
# direction = "both" allows both forward selection and backward elimination
# The algorithm chooses the best subset of predictors based on AIC (Akaike Information Criterion)

summary(ols_fwdsw)
# Show the summary of the final model chosen by stepwise selection:
# includes coefficients, p-values, R², and diagnostic metrics

This method automatically selects a simpler, more efficient model by removing or adding predictors that improve the model's AIC. It's useful when you're unsure which combination of predictors performs best.

In [ ]:
# install.packages("estimatr")  # (Run once) For robust regression tools
library(estimatr)  # Load for lm_robust if needed

# install.packages("modelsummary")  # (Optional) For clean regression output
# library(modelsummary)

# (Optional robust version using heteroskedasticity-consistent SEs)
# ols_fwd_stepwise <- lm_robust(
#     total_points ~ assists + creativity + expected_goals + goals_scored +
#       influence + minutes + own_goals + penalties_missed + selected +
#       starts + threat + transfers_balance + value, data = db03, se_type = "stata")

# Standard OLS regression with predictors selected from stepwise procedure
ols_fwd_stepwise <- lm(
  total_points ~ assists + creativity + expected_goals + goals_scored +
    influence + minutes + own_goals + penalties_missed + selected +
    starts + threat + value, data = db03)

summary(ols_fwd_stepwise)
# View the summary: coefficients, significance levels, R², and diagnostics

# (Optional) Produce a nicely formatted regression table
# modelsummary(ols_fwd_stepwise, stars = TRUE)

This model is leaner, based on variables chosen by the stepwise AIC approach — keeping only those that statistically and economically contribute to explaining total_points.

In [ ]:
# install.packages("MuMIn")  # (Run once) Install 'MuMIn' for model selection tools
library(MuMIn)  # Load the MuMIn package

options(na.action = "na.fail")
# Required by 'dredge': forces R to fail if missing data exists (ensures full variable combinations are valid)

ols_mumin <- get.models(
  dredge(ols_fwd_base, rank = "AICc"), 1
)[[1]]
# Perform exhaustive model search using 'dredge', ranked by AICc (corrected AIC for small samples)
# 'get.models(..., 1)[[1]]' extracts the **best model** (lowest AICc) from the full model set

summary(ols_mumin)
# Display the summary of the best model: coefficients, R², significance levels, and residuals

This approach performs an automated model selection across all possible combinations of predictors, not just forward/backward paths — using AICc as the selection criterion.

In [ ]:
# install.packages("estimatr")  # (Run once) Install estimatr for robust regression
library(estimatr)  # Load estimatr (if using lm_robust)

# install.packages("modelsummary")  # (Optional) Install for nice summary tables
# library(modelsummary)

# Robust version (commented out): for heteroskedasticity-consistent standard errors
# ols_fwd_dredge <- lm_robust(
#     total_points ~ assists + creativity + expected_goals + goals_scored +
#       influence + minutes + own_goals + penalties_missed + selected +
#       starts + threat + value, data = db03, se_type = "stata")

# Standard OLS model with predictors chosen via MuMIn::dredge (best AICc model)
ols_fwd_dredge <- lm(
  total_points ~ assists + creativity + expected_goals + goals_scored +
    influence + minutes + own_goals + penalties_missed + selected +
    starts + threat + value, data = db03)

summary(ols_fwd_dredge)
# View regression output: coefficient estimates, significance, R², residual stats

# (Optional) Clean output table with significance stars
# modelsummary(ols_fwd_dredge, stars = TRUE)

This final model reflects the most AICc-efficient subset from your original full model, selected via MuMIn::dredge.

In [ ]:
# install.packages("olsrr")  # (Run once) Install 'olsrr' for stepwise regression diagnostics
library(olsrr)  # Load the olsrr package

ols_step_both_p(ols_fwd_base)
# Perform stepwise regression based on p-values:
# - Starts with the full model (ols_fwd_base)
# - Adds/removes variables one at a time
# - Chooses variables based on their statistical significance (p-value thresholds)
# - Stops when no further significant improvements can be made

# Output:
# - A step-by-step log of which variables were added or removed
# - Final model summary (AIC, R², etc.)

This gives a p-value-based alternative to AIC-based stepwise methods — useful for quick, interpretable model refinement based on significance thresholds.

In [ ]:
# install.packages("estimatr")  # (Run once) Install for robust standard error options
library(estimatr)  # Load for lm_robust (optional robust regression)

# install.packages("modelsummary")  # (Optional) Install for clean summary tables
# library(modelsummary)

# (Optional) Robust OLS version using heteroskedasticity-consistent SEs
# ols_fwd_olsrr <- lm_robust(
#     total_points ~ influence + assists + goals_scored + minutes + penalties_missed +
#       threat + starts + own_goals + creativity + value +
#       expected_goals + selected, data = db03, se_type = "stata")

# Standard OLS model using predictors selected via olsrr::ols_step_both_p()
ols_fwd_olsrr <- lm(
  total_points ~ influence + assists + goals_scored + minutes + penalties_missed +
    threat + starts + own_goals + creativity + value +
    expected_goals + selected, data = db03)

summary(ols_fwd_olsrr)
# Show regression summary: estimates, standard errors, significance, and fit stats

# (Optional) Display a clean summary table with significance stars
# modelsummary(ols_fwd_olsrr, stars = TRUE)

This model uses predictors identified via p-value-based stepwise selection `(ols_step_both_p())`, giving you a practical and statistically guided subset.

In [ ]:
# install.packages("gridExtra")  # (Run once) For arranging multiple plots
library(gridExtra)  # Load for grid-based plot arrangement

# install.packages("sjPlot")  # (Run once) For plotting model summaries
library(sjPlot)  # Load for visualizing regression results

p1 <- plot_model(ols_fwd_dredge,           # Use the model selected via MuMIn::dredge
                 type = "est",             # Plot unstandardized coefficient estimates
                 show.values = TRUE,       # Show the coefficient values next to the bars
                 value.offset = 0.3,       # Offset distance for the displayed values
                 title = "OLSrr: Estimated",  # Title for the plot
                 vline.color = "gray50") +    # Color of the vertical zero reference line
  theme_minimal()  # Apply a clean minimal theme

# Optional standardized version (commented out)
# p2 <- plot_model(ols_fwd_dredge,
#                  type = "std",           # Plot standardized coefficients (beta)
#                  show.values = TRUE,
#                  value.offset = 0.3,
#                  title = "OLSrr: Standardized",
#                  vline.color = "gray50") +
#   theme_minimal()

grid.arrange(p1, ncol = 1)  # Display the plot(s) in a grid layout (here: just p1 in 1 column)

This generates a visual summary of your regression coefficients, making it easier to interpret direction, size, and confidence intervals at a glance.

In [ ]:
install.packages("forestmodel")  # (Run once) Install the 'forestmodel' package for forest-style coefficient plots

library(forestmodel)  # Load the package

forest_model(
  ols_fwd_dredge,  # Use the final OLS model selected via MuMIn::dredge
  theme = theme_forest(),  # Apply a clean, publication-ready forest plot theme
  format_options = forest_model_format_options(text_size = 4)  # Adjust font size (smaller for compact display)
)

This creates a forest plot showing:
- Coefficient estimates
- 95% confidence intervals
- Significance visually (whether CI crosses zero)

In [ ]:
install.packages("see")  # (Run once) Install 'see' for plotting model diagnostics

library(performance)  # Load for model checking tools (residuals, multicollinearity, etc.)
library(see)          # Load for visualizing performance checks

check_model(ols_fwd_dredge)
# Automatically generates a panel of diagnostic plots:
# Residuals vs Fitted
# Normal Q-Q
# Scale-Location
# Cook’s Distance
# Leverage
# Multicollinearity (VIF)

# Great for quickly checking:
# - Linearity
# - Homoscedasticity
# - Influential observations
# - Normality of residuals
# - Multicollinearity

**A simple guide to interpreting the plots from `check_model()` in plain English:**

***Residuals vs Fitted***

> What to look for: Points should be randomly scattered around the horizontal line (y = 0).
> If you see a pattern (curve or funnel):
- Your model may have non-linearity or heteroskedasticity.
- Consider transforming variables or using a different model.


***Normal Q-Q (Quantile–Quantile Plot)***

> What to look for: Points should follow the diagonal line.
> If points deviate a lot at the ends:
- Your residuals may not be normally distributed.
- Normality matters most for inference (p-values, confidence intervals).

***Scale-Location (Spread–Location Plot)***

> What to look for: Points should be randomly spread with a flat trend.
> If it fans out or has a pattern:
- Your model may suffer from non-constant variance (heteroskedasticity).

***Cook’s Distance***

> What to look for: Most points should be low and similar in height.
- Tall spikes: Indicate influential points — data that heavily affects the model.
- Investigate these — they may be valid outliers or data entry issues.

***Leverage Plot***

> What to look for: Most points should be close to the left.
- Points far to the right: These have high leverage (unusual x-values).
- If also high in Cook’s Distance → potentially problematic outlier.

***Multicollinearity (VIF)***

> What to look for: VIF values should ideally be < 5.
- If VIF > 5–10: Suggests high multicollinearity — predictors may be too correlated.
- Consider removing or combining variables.

In [ ]:
# Required packages
# install.packages("car")       # (Run once) Provides tools like VIF and linearHypothesis
# install.packages("lmtest")    # (Run once) Provides tests like Breusch–Pagan and Durbin–Watson

library(car)     # Load the 'car' package for regression diagnostics
library(lmtest)  # Load the 'lmtest' package for hypothesis testing and autocorrelation checks

# ---- 1. Model Summary ----
cat("\n Model Summary:\n")
print(summary(ols_fwd_dredge))  # Display coefficients, R-squared, and p-values for your model

# ---- 2. Multicollinearity Check ----
cat("\n Variance Inflation Factor (VIF):\n")
print(vif(ols_fwd_dredge))  # Check if any predictors are too correlated (VIF > 5 or 10 is a concern)

# ---- 3. Heteroskedasticity Test (Breusch–Pagan) ----
cat("\n Breusch–Pagan Test for Heteroskedasticity:\n")
print(bptest(ols_fwd_dredge))  # Test whether residual variance is constant
# If p < 0.05 → your model may have heteroskedasticity (bad)

# ---- 4. Autocorrelation Check (Durbin-Watson) ----
cat("\n Durbin–Watson Test for Autocorrelation:\n")
print(dwtest(ols_fwd_dredge))  # Test if residuals are correlated (esp. in time series)
# DW ~ 2 is ideal. Much < 2 suggests positive autocorrelation

# ---- 5. Normality of Residuals ----
cat("\n Shapiro-Wilk Test for Normality of Residuals:\n")
print(shapiro.test(residuals(ols_fwd_dredge)))  # Test if residuals are normally distributed
# p > 0.05 means residuals are likely normal (a good thing)

# # ---- 6. Influence and Outlier Detection ----
# cat("\n Influential Observations (Cook's Distance > 4/n):\n")
# cooks_d <- cooks.distance(ols_fwd_dredge)        # Measure how much each observation influences the model
# n <- length(cooks_d)                             # Get the number of observations
# influential_obs <- which(cooks_d > (4 / n))      # Flag observations with high influence
# print(influential_obs)                           # Show their row numbers

# # ---- 7. Joint Hypothesis Test (All team dummies = 0) ----
# team_vars <- grep("^team", names(coef(ols_fwd_dredge)), value = TRUE)  # Find all team dummy variables
# cat("\n Joint Significance Test for All Team Variables:\n")
# print(linearHypothesis(ols_fwd_dredge, team_vars))  # Test if all team effects = 0 at once

# ---- 8. Residual Plots (Optional) ----
par(mfrow = c(2, 2))     # Arrange 4 plots in one 2x2 layout
plot(ols_fwd_dredge)     # Generate diagnostic plots: residuals, Q-Q, leverage, etc.
par(mfrow = c(1, 1))     # Reset to default plotting layout

In [ ]:
library(car)  # Load the 'car' package to use the VIF (Variance Inflation Factor) function

# Define a function to reduce multicollinearity by stepwise VIF elimination
vif_stepwise <- function(model, thresh = 5, trace = TRUE) {

  vifs <- vif(model)  # Calculate initial VIFs for all predictors

  while (any(vifs > thresh)) {  # Keep looping as long as at least one VIF exceeds the threshold

    var_to_drop <- names(which.max(vifs))  # Find the variable with the highest VIF

    if (trace) {
      cat("Dropping:", var_to_drop, "| VIF =", max(vifs), "\n")  # Print which variable is being dropped
    }

    fmla <- formula(model)  # Extract the model formula
    fmla <- update(fmla, paste(". ~ . -", var_to_drop))  # Update the formula by removing the high-VIF variable

    model <- lm(fmla, data = model$model)  # Refit the model with the reduced formula
    vifs <- vif(model)  # Recalculate VIFs
  }

  return(model)  # Return the final model with all VIFs below the threshold
}

What this function does:
- Automatically removes the most collinear variable one at a time (based on highest VIF).
- Stops when all remaining VIFs are below the given threshold (default = 5).
- Prints progress if trace = TRUE.

In [ ]:
# Start with full model (commented out here — assumed already defined)
# ols_fwd_dredge <- lm(
#     total_points ~ assists + creativity + expected_goals + goals_scored +
#       influence + minutes + own_goals + penalties_missed + selected +
#       starts + threat + value, data = db03)

# Run VIF-guided backward selection
ols_fwd_treated <- vif_stepwise(ols_fwd_dredge, thresh = 5)
# This automatically drops predictors one by one if their VIF > 5,
# reducing multicollinearity from the model

# Review the refined model
summary(ols_fwd_treated)
# Shows the final model after removing collinear predictors:
# includes coefficients, significance, and fit metrics

In [ ]:
library(car)      # For calculating Variance Inflation Factor (VIF)
library(tibble)   # For tidy conversion of row names into a column
library(dplyr)    # For data manipulation like sorting and renaming

# Calculate VIF and convert result into a clean tibble
vif_table <- vif(ols_fwd_treated) %>%          # Compute VIFs for all predictors in the treated model
  as.data.frame() %>%                          # Convert the named vector to a data frame
  rownames_to_column(var = "Variable") %>%     # Move variable names from row names to a column
  rename(VIF = ".")                            # Rename the automatically generated column "." to "VIF"

# Optionally sort variables by descending VIF value
vif_table <- vif_table %>%
  arrange(desc(VIF))

# Print the final clean VIF table
print(vif_table)

In [ ]:
# install.packages("estimatr")  # (Run once) For robust standard errors
library(estimatr)  # Load if using lm_robust

# install.packages("modelsummary")  # (Optional) For clean regression tables
# library(modelsummary)

# (Optional robust version with heteroskedasticity-consistent standard errors)
# ols_fwd_trtd <- lm_robust(
#     total_points ~ assists + creativity + expected_goals +
#     goals_scored + own_goals + penalties_missed +
#     selected + starts + threat + value, data = db03, se_type = "stata")

# Standard OLS regression using final, low-multicollinearity predictors
ols_fwd_trtd <- lm(
  total_points ~ assists + creativity + expected_goals +
    goals_scored + own_goals + penalties_missed +
    selected + starts + threat + value, data = db03)

summary(ols_fwd_trtd)
# Print model summary: coefficients, p-values, R², and diagnostic info

# Optional: create a clean, publication-style summary table
# modelsummary(ols_fwd_trtd, stars = TRUE)

In [ ]:
library(forestmodel)  # Load the package to create forest plots for regression models

# Create forest plot object from final OLS model
p_forest <- forest_model(
  ols_fwd_trtd,                         # Use the VIF-treated regression model
  theme = theme_forest(),              # Apply a clean forest-style theme
  format_options = forest_model_format_options(text_size = 4)  # Set small font for compact display
)

# Save the plot to a PDF file
pdf("/content/drive/MyDrive/00fpl/forest_plot.pdf", width = 11.69, height = 8.27)
# Open a PDF device (A4 landscape size) to save the plot

# Arrange and render the plot
grid.arrange(
  grobs = list(p_forest),  # Put the forest plot in a list (can add more plots later)
  ncol = 1                  # Arrange in 1 column (i.e., single full-page plot)
)

dev.off()  # Close the PDF device and finalize the file

In [ ]:
# install.packages("gridExtra")  # (Run once) For arranging plots
library(gridExtra)  # Load to use grid.arrange()

# install.packages("sjPlot")  # (Run once) For visualizing regression models
library(sjPlot)  # Load to use plot_model()

# Create a standardized coefficient plot
p_sj <- plot_model(
  ols_fwd_trtd,                    # Use the final VIF-treated OLS model
  type = "std",                    # Plot standardized beta coefficients
  show.values = TRUE,              # Display coefficient values on the plot
  value.offset = 0.3,              # Move the text slightly away from the bars
  title = "OLSrr: Standardized",  # Title for the plot
  vline.color = "gray50"           # Vertical reference line at zero
) +
  theme_minimal()                  # Apply a clean theme

# Save the plot as a landscape A4 PDF
pdf("/content/drive/MyDrive/00fpl/sj_plot.pdf", width = 11.69, height = 8.27)
# Open a PDF device to write the plot

# Render the plot into the PDF
grid.arrange(
  grobs = list(p_sj),  # Place the plot into a list of grobs (plot objects)
  ncol = 1              # Arrange in 1 column
)

dev.off()  # Finalize and close the PDF device

In [ ]:
# install.packages("stargazer")  # (Run once) Install stargazer for beautiful regression tables
library(stargazer)  # Load stargazer for formatting regression output

# Export a side-by-side regression table for 3 models
stargazer(
  ols_fwd_stepwise,                # Model from stepwise selection
  ols_fwd_dredge,                  # Model from AICc selection via MuMIn::dredge
  ols_fwd_trtd,                    # Final model after VIF filtering
  type = "text",                   # Output format: plain text (other options: "html", "latex")
  out = "/content/drive/MyDrive/00fpl/ols_fwd_estimatr.txt"  # Save output to this text file
)